In [ ]:
# Install bob
# -------------------------------------------------------
# -------------------------------------------------------
!pip install gym keras keras-rl2 tensorflow==2.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import os
import pandas as pd 
from datetime import datetime as dt
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
from numpy import loadtxt
from datetime import datetime
from numpy import savetxt
import random ; from random import randint
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense , Flatten 
from tensorflow.keras.optimizers import Adam
from rl.policy import * ; from rl.memory import * ; from rl.agents import *

In [ ]:
# Global Varaile
# -------------------------------------------------------
# -------------------------------------------------------
Memoryin_window_length      = 10
window_width                = 64
window_height               = 32
ob_space_Length             = 3
actions                     = 3
MemoryLimit                 = 10000
#-----------------------------------------------------------
fee                         = 1
SL                          = 20
TP                          = 20
#------------------------------------------------------------
next_trade_one_day          = 10
nb_trade_one_day            = 20
nb_barsM1_for_testing_res   =1300*2 

In [ ]:
# fun
# -------------------------------------------------------

def get_local_data():
	df = pd.read_csv ('/content/drive/MyDrive/DATA/2020.csv',encoding='utf-16',header=None, names=['Date', 'Open','High','Low','Close'])
	return df

def get_matrix( width , height ):	 
	#arr      = np.empty((height,width), dtype=object);marke    = ' '
	arr      = np.empty((height,width), dtype=np.float16 );marke    = 0
	for i in range(0,height):
		for j in range(0,width): 
			arr[i,j] = marke
	return arr  
def print_matrix(m):
	line = '\n array :(' + str( len(m) ) + ',' + str( len(m[0]) ) + ')'
	for i in range( 0 , len(m) ): 
		line += '\n'
		for j in range(0, len(m[0]) ): 
			line += str( m[i,j]  )	
	print(line)
   
# -------------------------------------------------------
def plot_line_v2(line,ax,ay):
    x  = line
    y = np.arange(0,len(x) )
    plt.figure( figsize=(ax,ay) )
    plt.title('Simple Graphe')
    plt.xlabel('Date',fontsize=18)
    plt.ylabel('Prices USD',fontsize=18)
    plt.plot(  y , x )
    plt.legend( [ 'Open' ,  'Close' ] ,loc='lower right' )
    plt.show()    
#---------------------------------------------------------     
def reshape(v,window_height): #40
    max_ = max(v)
    min_ = min(v)
    res = (v - min_)/(max_-min_)
    res =  np.round( res , 2 )*window_height
    res = res.astype(int) 
    return res
#---------------------------------------------------------    
def get_micro(price,window_height):
    reshape_price = reshape ( price , window_height )
    new = get_matrix( len(price) ,window_height)
    for i in range(0, new.shape[1] ): 
        G = window_height - reshape_price[i] - 1
        new[ G ,  i  ] = 0.5
    #new = new.astype(np.float32)    
    return new
#--------------------------------------------------------- 
def get_data( dataset, period_current, iTime_index, range_bars ):
    res      = np.array( data[iTime_index-range_bars*period_current:iTime_index] )
    res      = res[:,1] # Close
    if(period_current>1):
        Bg_res = res
        res =  np.empty(range_bars, dtype=np.float32)
        i_res = 0
        for i in range( 0, len(Bg_res) ):
            #print( ' i = ',i,' mod =  ' , i%period_current , ' val = ',Bg_res[i] )
            if(i%period_current==0):
                res[i_res] = Bg_res[i]
                i_res +=1
    return res   
# ================================================================
def isShort( indexStart ): 
  global data , TP , SL , fee
  prices = get_data( data , 1  , indexStart , nb_barsM1_for_testing_res )
  res = -1
  for i in range(0,nb_barsM1_for_testing_res):
    if( prices[i] <= (prices[0]-TP) ):
      res = TP - fee
      break
    if( prices[i] >= (prices[0]+SL) ):
      res = -SL 
      break
  return res 
def isLong( indexStart ):
  global data , TP , SL , fee
  prices = get_data( data , 1  , indexStart , nb_barsM1_for_testing_res )
  res = -1
  for i in range(0,nb_barsM1_for_testing_res):
    if( prices[i] >= (prices[0]+TP) ):
      res = TP - fee
      break
    if( prices[i] <= (prices[0]-SL) ):
      res = -SL 
      break
  return res
def show():
  global AllReward
  v = 0
  v_v = []
  for i in range(0,len(AllReward)):
    v += AllReward[i]
    v_v.append(v)
  print( ' AllReward = ',v_v[len(v_v)-1] )
  plot_line_v2(v_v,18,7)


In [ ]:
# Get Data 
data =   pd.read_csv('/content/drive/MyDrive/DATA/US30_from2021.csv',encoding='utf-16',header=None, names=['Date', 'Open','High','Low','Close'])
data

,Date,Open,High,Low,Close
0,1609459140,30603.0,NaN,NaN,NaN
1,1609722000,30606.0,NaN,NaN,NaN
2,1609722060,30595.0,NaN,NaN,NaN
3,1609722120,30569.0,NaN,NaN,NaN
4,1609722180,30573.0,NaN,NaN,NaN
...,...,...,...,...,...
105864,1619049240,34110.0,NaN,NaN,NaN
105865,1619049300,34108.0,NaN,NaN,NaN
105866,1619049360,34110.0,NaN,NaN,NaN
105867,1619049420,34109.0,NaN,NaN,NaN


In [ ]:
test_data = get_data( data,1,19000,window_width )
#d = get_micro(test_data , window_height )
#print_matrix(d)
#plot_line_v2(test_data,7,12)
print( ' isLong = ',isLong(19000) )
test_data = get_data( data,1,19000 + window_width ,window_width + nb_barsM1_for_testing_res )
#d = get_micro(test_data , window_height )
#print_matrix(d)
#plot_line_v2(test_data,10,12)

 isLong =  -20.0


In [ ]:
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
#---------------     gym environment           -------------------------------------
#-----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------
class envDRL4K(Env ):
    def __init__(self , iTime_start ):
         #------------  observation_space  observation_space --------------------------------------------
        global data , ob_space_Length , window_width , window_height , TP , SL , fee  , next_trade_one_day , nb_trade_one_day , AllReward
        self.window                     = window_width   # 360Min = 6H (Best View) 
        self.observation_space          = Box(low= 0.0, high= 1.0, shape=( ob_space_Length , window_width , window_height), dtype=np.float16 )
        self.action_space               = Discrete(3)

        #------------  Parameters --------------------------------------------
        self.iTime                      = iTime_start            
        self.iTime_one_day              = 0       
        self.nb_trade_one_day           = nb_trade_one_day
        self.SL                         = SL
        self.TP                         = TP

        self.D_first                    = get_micro(  get_data( data , 1  , self.iTime , self.window ) , window_height )
        self.D_second                   = get_micro(  get_data( data , 5  , self.iTime , self.window ) , window_height )
        self.D_third                    = get_micro(  get_data( data , 15 , self.iTime , self.window ) , window_height )
        
        self.state                      = np.array([  self.D_first , self.D_second , self.D_third ])
        print( ' environment is creat' )
    def step(self, action):
        reward = 0
        if(action==0):# sell
            reward = isShort(self.iTime)
            AllReward.append(reward)
    

        if(action==1):# Buy
            reward = isLong(self.iTime)
            AllReward.append(reward)

        if(action==2):# none
            reward = 0
         
        
        info_ = {}
        done = False
        self.iTime_one_day       += 1
        self.iTime               += next_trade_one_day
        if( self.iTime_one_day >=self.nb_trade_one_day ):  # making 100 Trades one day
            done = True
        if( self.iTime>= len(data)  ):                     # finishing All data
            self.iTime = iTime_start 
            done = True
            
        self.D_first                    = get_micro(  get_data( data , 1  , self.iTime , self.window ) , window_height )
        self.D_second                   = get_micro(  get_data( data , 5  , self.iTime , self.window ) , window_height )
        self.D_third                    = get_micro(  get_data( data , 15 , self.iTime , self.window ) , window_height )
        
        self.state                          = np.array([  self.D_first , self.D_second , self.D_third ])
        return self.state, reward, done, info_
    
    def render(self):
        Rv = 0
        
    def reset(self):
        # Reset shower temperature
        
        self.iTime               += random.randint(60*2, 60*4)
        self.iTime_one_day = 0
            
        if( self.iTime>= len(data)  ):
            self.iTime = iTime_start
            self.iTime_one_day = 0
        
        self.D_first                    = get_micro(  get_data( data , 1  , self.iTime , self.window ) , window_height )
        self.D_second                   = get_micro(  get_data( data , 5  , self.iTime , self.window ) , window_height )
        self.D_third                    = get_micro(  get_data( data , 15 , self.iTime , self.window ) , window_height )
        
        self.state                          = np.array([  self.D_first , self.D_second , self.D_third ])
        
        print( self.iTime )

        return self.state
#-----------------------------------------------------------------
env = envDRL4K(12000)

 environment is creat


In [ ]:
AllReward = []
env = envDRL4K(12000)
for i_episode in range(10):
    observation = env.reset()
    Allreward = 0
    for t in range(100):
        env.render()
        #print(' observation shape ',observation.shape)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        Allreward +=reward
        if done:
            print("Episode finished after {} timesteps __ reward = {}".format(t+1,Allreward))
            Allreward = 0
            break
env.close()

 environment is creat
Episode finished after 20 timesteps __ reward = -38.0
Episode finished after 20 timesteps __ reward = -21.0
Episode finished after 20 timesteps __ reward = -98.0
Episode finished after 20 timesteps __ reward = -81.0
Episode finished after 20 timesteps __ reward = -135.0
Episode finished after 20 timesteps __ reward = -78.0
Episode finished after 20 timesteps __ reward = -35.0
Episode finished after 20 timesteps __ reward = -18.0
Episode finished after 20 timesteps __ reward = -58.0
Episode finished after 20 timesteps __ reward = 5.0


> **Mode DQN****


In [ ]:
def build_model(ob_space_Length , window_width , window_height, actions,Memoryin_window_length):
    model = Sequential()
    model.add(Dense(128,activation='relu', input_shape=(Memoryin_window_length,ob_space_Length  , window_height , window_width) ) )
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Flatten())
    model.add(Dense(actions))
    return model    
def build_agent(model, actions,Memoryin_window_length):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=1000)
    memory = SequentialMemory(limit= MemoryLimit , window_length=Memoryin_window_length)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000 ,
                   gamma=0.95 
                  )
    return dqn  
model = build_model(ob_space_Length , window_width , window_height, actions,Memoryin_window_length)
model.summary()
dqn = build_agent(model, actions,Memoryin_window_length)
dqn.compile( Adam(lr=1e-4))    
name  = '/content/drive/MyDrive/DATA/DRL_4K/07-42-14/07-42-14.h5f'
dqn.load_weights(name)    
        

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10, 3, 32, 128)    8320      
_________________________________________________________________
dense_7 (Dense)              (None, 10, 3, 32, 256)    33024     
_________________________________________________________________
dense_8 (Dense)              (None, 10, 3, 32, 256)    65792     
_________________________________________________________________
dense_9 (Dense)              (None, 10, 3, 32, 128)    32896     
_________________________________________________________________
flatten_1 (Flatten)          (None, 122880)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 368643    
Total params: 508,675
Trainable params: 508,675
Non-trainable params: 0
________________________________________________

In [ ]:
next_trade_one_day          = 10
nb_trade_one_day            = 300
AllReward = []
env = envDRL4K(4000)
_ = dqn.test(env, nb_episodes=6  , visualize=False)
show()

In [ ]:
show()

In [ ]:
env = envDRL4K(12000)
FileVersion = "DRL_4K/"
time = datetime.now()
time = time.strftime("%H-%M-%S")
steps = 400000
Looping = 1
NameSaving = "testing"
for i in range(0,Looping):
  dqn.fit(env, nb_steps=steps, visualize=False, verbose=1)
  time_ = datetime.now()
  time_ = time_.strftime("%H-%M-%S")
  NameSaving = FileVersion+time_+'/' +str(time_)+".h5f"
  print('IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII') 
  print(NameSaving)
  print('IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII')
  dqn.save_weights(NameSaving, overwrite=True)

 environment is creat
Training for 400000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 11087s 1s/step - reward: -0.4589
250 episodes - episode_reward: -18.356 [-1512.000, 1598.000] - loss: 105.657 - mean_q: 17.786 - mean_eps: 0.100

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 12646s 1s/step - reward: -0.6808
250 episodes - episode_reward: -27.232 [-912.000, 1025.000] - loss: 102.941 - mean_q: 30.463 - mean_eps: 0.100

Interval 3 (20000 steps performed)
 3819/10000 [==========>...................] - ETA: 2:21:34 - reward: -0.0545done, took 28986.105 seconds
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
DRL_4K/20-35-14/20-35-14.h5f
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
